In [1]:
import numpy as np
import pandas as pd

from fbprophet import Prophet

from keras.layers import Dense, Activation, GRU, Dropout
from keras.models import Sequential

from tensorflow.python.client import device_lib
import tensorflow as tf

from keras import backend

Using TensorFlow backend.


In [2]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)

In [3]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True, gpu_options=gpu_options))
print(device_lib.list_local_devices())
backend.tensorflow_backend._get_available_gpus()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17704761148460219608
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11839838617
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13452342969260261690
physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:1e.0, compute capability: 7.0"
]


['/job:localhost/replica:0/task:0/device:GPU:0']

In [21]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
(tf.Session(config=config))

In [4]:
etf_frame = pd.read_csv('/home/ubuntu/csvs/etfs_pivot.csv')
etfs = etf_frame.columns.tolist()

In [5]:
etfs.pop(0)

'date'

In [6]:
train_date = pd.date_range(start='2014-03-17', end='2017-03-15')
macro_date = pd.date_range(start='2016-03-17', end='2017-03-15')
val_date = pd.date_range(start='2017-03-17', end='2018-03-15')
test_date = pd.date_range(start='2018-03-17', end='2019-03-15')

In [7]:
macro = pd.read_csv('/home/ubuntu/csvs/macro_pivot.csv')

In [8]:
etf_frame = etf_frame.set_index('date')
macro = macro.set_index('date')

etf_frame.index = pd.to_datetime(etf_frame.index)
macro.index = pd.to_datetime(macro.index)

In [9]:
predictions = pd.DataFrame(index=etf_frame.index)

In [10]:
X_frame = macro[macro.index.isin(macro_date)]
X_array = X_frame.values
X = np.reshape(X_array, (X_array.shape[0], X_array.shape[1], 1))

In [11]:
def make_prediction(etf):
    single_etf = etf_frame[etf]
    single_etf = single_etf.reset_index()
    single_etf.columns = ['ds','y']
    
    arima = Prophet()
    arima.fit(single_etf[single_etf['ds'].isin(train_date)])
    future = arima.make_future_dataframe(periods=len(val_date))
    forecast = arima.predict(future)
    
    residuals = single_etf[single_etf['ds'].isin(val_date)]['y'] - forecast[forecast['ds'].isin(val_date)]['yhat']
    y = residuals
    y = np.reshape(y, (y.shape[0], 1))
    gru_model = Sequential()
    
    gru_model.add(GRU(units=100, input_shape=(28779,1), return_sequences=True))
    gru_model.add(Dropout(0.2))
    gru_model.add(GRU(units=50, return_sequences=False))
    gru_model.add(Dropout(0.2))
    gru_model.add(Dense(1))
    gru_model.add(Activation('linear', input_shape=(365,1)))
    
    gru_model.compile(loss='mse', optimizer='rmsprop')
    gru_model.fit(x=X,y=y, batch_size=512, epochs = 5, validation_split=0.25)
    
    new_future = arima.make_future_dataframe(periods=(len(val_date)+len(test_date)))
    new_forecast = arima.predict(future)
    
    new_X_array = macro[macro.index.isin(val_date)].values
    new_X = np.reshape(X_array, (X_array.shape[0], X_array.shape[1], 1))
    
    new_residuals = gru_model.predict(new_X)
    arima_prediction = arima[arima['ds'].isin(test_date)]['yhat']
    
    return arima_prediction + new_residuals

In [12]:
etf_risky = ['BIB', 'BRZU', 'CURE', 'DRN', 'EDC', 'FAS', 'FINU', 'GOEX', 'INDL','JNUG', 'JPNL', 'KOLD', 'LBJ',
             'MIDU', 'NUGT', 'QLD', 'RETL', 'ROM','SCO', 'SOXL', 'SPXL', 'SVXY', 'TECL', 'TMF', 'TNA', 'TQQQ',
             'UBR','UDOW', 'UMDD', 'UPRO', 'URTY', 'USD', 'UWM', 'XPP', 'YINN']

In [22]:
for etf in etf_risky:
    
    prediction = make_prediction(etf)
    
    predictions[etf] = prediction

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


Train on 273 samples, validate on 91 samples
Epoch 1/5


InternalError: GPU sync failed